In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import requests
import time

In [2]:
# Download Example
url_ex = 'https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41.nc?analysed_sst%5B(2002-06-01T09:00:00Z)%5D%5B(-89.99):1000:(89.99)%5D%5B(-179.99):1000:(180.0)%5D'

nc = requests.get(url_ex)

with open(r'D:\python_gis\week6\notebooks_w6\aa.nc', 'wb') as f:
    f.write(nc.content)

In [3]:
# Define gfs Download setting
'''
Index Information: https://pae-paha.pacioos.hawaii.edu/erddap/griddap/ncep_global.html
===================================
tmpsfc (surface air temperature, K) 
tmp2m (air temperature at 2m, K) 
ugrd10m (eastward wind velocity at 10m, m/s) 
vgrd10m (northward wind velocity at 10m, m/s) 
pratesfc (rainfall rate, kg m-2 s-1) 
rh2m (relative humidity at 2m, %) 
prmslmsl (mean sea level pressure, Pa) 
dlwrfsfc (net downward longwave radiation flux, W m-2) 
dswrfsfc (net downward shortwave radiation flux, W m-2) 
'''

condition_dict = {'tmpsfc': True,
                  'tmp2m': True,
                  'ugrd10m': True,
                  'vgrd10m':True,
                  'pratesfc':True,
                  'rh2m':True,
                  'prmslmsl':True,
                  'dlwrfsfc':True, 
                  'dswrfsfc':True,
                  'start_date':'(2022-04-25T12:00:00Z)',
                  'end_date':'(2022-04-30T12:00:00Z)',
                  'lat_range':[0,30],
                  'long_range':[118,150]
                 }


# Change setting inside condition_dict directly or add lines as below
# condition_dict["tmpsfc"] = False

print(condition_dict)

{'tmpsfc': True, 'tmp2m': True, 'ugrd10m': True, 'vgrd10m': True, 'pratesfc': True, 'rh2m': True, 'prmslmsl': True, 'dlwrfsfc': True, 'dswrfsfc': True, 'start_date': '(2022-04-25T12:00:00Z)', 'end_date': '(2022-04-30T12:00:00Z)', 'lat_range': [0, 30], 'long_range': [118, 150]}


In [4]:
import os
import time

save_dir = r'D:\python_gis\week6\notebooks_w6' #change to your saving oath

def define_url(condition_dict):
    
    output_url = ''
    
    # header url part
    org_url = 'https://pae-paha.pacioos.hawaii.edu/erddap/griddap/ncep_global.nc?'
    # Generate condition url part
    START_DATE = condition_dict.get('start_date')
    END_DATE = condition_dict.get('end_date')
    MIN_LAT = str(condition_dict.get('lat_range')[0])
    MAX_LAT = str(condition_dict.get('lat_range')[1])
    MIN_LONG = str(condition_dict.get('long_range')[0])
    MAX_LONG = str(condition_dict.get('long_range')[1])
    
    cond_url_date = '%5B' + START_DATE + ':1:' + END_DATE 
    cond_url_loc = '%5D%5B(' + MIN_LAT + '):1:(' + MAX_LAT+')%5D%5B(' + MIN_LONG + '):1:(' + MAX_LONG + ')%5D'
    cond_full_url = cond_url_date + cond_url_loc
    
    filename = define_filename(cond_full_url)
    
    # If statement to add url
    First_index_added = False # Inorder to decide whether or not add ',' in front of index, only the first index after .nc?
                              # doesn't need to add ','
     
    keys_list = [*condition_dict]  # get list of key name of dict
    
    # first 9 index into loop for condition check
    for i in range(9):
        
        if condition_dict.get(keys_list[i]):
            if First_index_added:
                output_url += ',' + keys_list[i] + cond_full_url  # since first index involved, just add ','
            else:
                First_index_added = True
                output_url += org_url + keys_list[i] + cond_full_url
                
    return output_url,filename

def define_filename(cond_full_url):
    
    # name = start_date_end_date_lat_range_long_range.nc
    name = cond_full_url.replace("%5D%5B","")
    name = name.replace(")%5D","")
    name = name.replace("%5B(","")
    name = name.replace(":1:","")
    name = name.replace(":00Z)","")
    name = name.replace("-","_")
    name = name.replace(":","_")
    name = name.replace("(","_")
    name = name.replace(")","_")
    name = name.replace("__","_")
    name = name + '_gfs.nc'
    return name


def download_url(url, filename, return_filepath = True):
    print("Request Status | Filename | Finish Time")
    t0 = time.time()
    try:
        r = requests.get(url)
        if r.status_code == requests.codes.ok:
            print("Request OK...",end=' ')
            file = os.path.join(save_dir,filename)
            with open(file, 'wb') as f:
                f.write(r.content)
                print(filename, 'in',round(time.time() - t0,2),'sec')
                return file
    
    except Exception as e:
        print('Exception in download_url():', e)

In [5]:
url,fn = define_url(condition_dict)
print('filename:',fn)

filename: 2022_04_25T12_00_2022_04_30T12_00_0_30_118_150_gfs.nc


In [6]:
download_file_path = download_url(url,fn)
print('File path:',download_file_path)

Request Status | Filename | Finish Time
Request OK... 2022_04_25T12_00_2022_04_30T12_00_0_30_118_150_gfs.nc in 16.51 sec
File path: D:\python_gis\week6\notebooks_w6\2022_04_25T12_00_2022_04_30T12_00_0_30_118_150_gfs.nc


In [7]:
# read nc file
#ds = xr.open_dataset(r"D:\python_gis\week6\notebooks_w6\aa.nc", engine="netcdf4")
ds = xr.open_dataset(download_file_path, engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:    (time: 41, latitude: 61, longitude: 65)
Coordinates:
  * time       (time) datetime64[ns] 2022-04-25T12:00:00 ... 2022-04-30T12:00:00
  * latitude   (latitude) float64 0.0 0.5 1.0 1.5 2.0 ... 28.5 29.0 29.5 30.0
  * longitude  (longitude) float64 118.0 118.5 119.0 119.5 ... 149.0 149.5 150.0
Data variables:
    tmpsfc     (time, latitude, longitude) float32 ...
    tmp2m      (time, latitude, longitude) float32 ...
    ugrd10m    (time, latitude, longitude) float32 ...
    vgrd10m    (time, latitude, longitude) float32 ...
    pratesfc   (time, latitude, longitude) float32 ...
    rh2m       (time, latitude, longitude) float32 ...
    prmslmsl   (time, latitude, longitude) float32 ...
    dlwrfsfc   (time, latitude, longitude) float32 ...
    dswrfsfc   (time, latitude, longitude) float32 ...
Attributes: (12/63)
    _CoordSysBuilder:           ucar.nc2.dataset.conv.CF1Convention
    acknowledgement:            The Pacific Islands Ocean Observing System (P...
    cdm_data_type:              Grid
    Conventions:                CF-1.6, ACDD-1.3
    creator_email:              ncep.list.pmb-dataflow@lstsrv.ncep.noaa.gov
    creator_institution:        NOAA National Centers for Environmental Predi...
    ...                         ...
    testOutOfDate:              now+136hours
    time_coverage_end:          2022-04-30T12:00:00Z
    time_coverage_resolution:   PT3H
    time_coverage_start:        2022-04-25T12:00:00Z
    title:                      NOAA/NCEP Global Forecast System (GFS) Atmosp...
    Westernmost_Easting:        118.0

In [8]:
import gc
ds.close()
gc.collect()

38

In [ ]:
# Apply to different dataset
# Example url : https://pae-paha.pacioos.hawaii.edu/erddap/griddap/dhw_5km_monthly.nc?CRW_DHW%5B(2021-12-01T12:00:00Z):1:(2021-12-01T12:00:00Z)%5D%5B(89.975):1:(-89.975)%5D%5B(-179.975):1:(179.975)%5D,CRW_HOTSPOT%5B(2021-12-01T12:00:00Z):1:(2021-12-01T12:00:00Z)%5D%5B(89.975):1:(-89.975)%5D%5B(-179.975):1:(179.975)%5D,CRW_SST%5B(2021-12-01T12:00:00Z):1:(2021-12-01T12:00:00Z)%5D%5B(89.975):1:(-89.975)%5D%5B(-179.975):1:(179.975)%5D
# Take NOAA Coral Reef Watch Monthly Global 5-km Satellite Coral Bleaching as example
# Step 1: change the org_url to 'https://pae-paha.pacioos.hawaii.edu/erddap/griddap/dhw_5km_monthly.nc?'

# Step 2: make a new condition_dict contian support indexes: CRW_DHW / CRW_HOTSPOT / CRW_SST,
# as well as start & end date, lat & long range

# Step 3: change line 33 number in range() to the number of all indexes of your url,i.e. change from 9 to 3 in this case.

# change line 57 output_name if you want,i.e. '_gfs.nc' to '_xxx.nc'
# Run the code
